**Generate codon-based alignments of all Scer strains (Peter, et al 2018) with European S. paradoxus (Bergström et al., 2014) and North American S. paradoxus subpop B (Durand et al., 2019) for input into the. McDonald-Kreitman test.**

In [1]:
import os
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
import re
from Bio.SeqRecord import SeqRecord
import pandas as pd
import subprocess
import sys
from os import system

### Make files: unaligned nucleotide and AA seqs for each gene, with all Scer strains and all Spar strains

In [3]:
#combine Scer and Spar nucleotide seqs

for filename in os.listdir('alignments/Spar_alignments/EuroSpar_NASpar_unaligned'):

    if filename[-3:] != '.fa':
        continue

    if os.path.exists('alignments/1011_all_strains_unaligned/'+filename+'sta'):
        call = 'cat alignments/Spar_alignments/EuroSpar_NASpar_unaligned/{} alignments/1011_all_strains_unaligned/{} > alignments/1011Scer_allSpar_codon_alignments/{}'.format(filename, filename+'sta', filename)
        system(call)

In [ ]:
#for each file, make an AA file from nucleotide sequence
#also write cleaner nucleotide file in same directory

comb_fastas = '/Users/clairedubin/sacc/carly_genes/alignments/1011Scer_allSpar_codon_alignments/nuc_unaligned_messy/'
nuc_outdir = '/Users/clairedubin/sacc/carly_genes/alignments/1011Scer_allSpar_codon_alignments/nuc_unaligned/'
aa_outdir = '/Users/clairedubin/sacc/carly_genes/alignments/1011Scer_allSpar_codon_alignments/AA_unaligned/'

for f in os.listdir(comb_fastas):

    if '.muscle_afa' in f:
        continue
        
    gene = f.strip('.fa')
    nuc_to_write = ''
    AA_to_write = ''
    
    for record in SeqIO.parse(comb_fastas+f, 'fasta'):
        
        aa_seq = record.seq.translate()
        
        nuc_to_write += '>' + record.id + '\n' + str(record.seq) + '\n'
        AA_to_write += '>' + record.id + '\n' + str(aa_seq) + '\n'
    
    nuc_out = open(nuc_outdir + gene + '.fna', 'w')
    nuc_out.write(nuc_to_write)
    nuc_out.close()
    
    aa_out = open(aa_outdir + gene + '.faa', 'w')
    aa_out.write(AA_to_write)
    aa_out.close()

NOTE: In some sequences there's a stop codon in the middle of a nucleotide sequence, which breaks PAL2NAL.

-if it happens in 10 or less strains within a gene, just drop those strains

-otherwise, drop the gene from analysis

-also drop genes with introns

In total, drop 929 genes, none of which are candidate genes.

In [2]:
intron_genes = []
for line in open('/Users/clairedubin/sacc/external_datasets/Scer.gff.txt', 'r').readlines():
    
    intron_count = line.split('introns=')[1][0]
    if intron_count != '0':
        intron_genes += [line.split('BLAST=')[1].split(';')[0]]
        

AA_dir = 'alignments/1011Scer_allSpar_codon_alignments/AA_unaligned/'
nuc_dir = 'alignments/1011Scer_allSpar_codon_alignments/nuc_unaligned/'

stops = {}
scer_stops = {}
spar_stops = {}
for i, AA_file in enumerate(os.listdir(AA_dir)):
    
    rl = open(AA_dir+AA_file, 'r').readlines()
    
    for line_num, line in enumerate(rl):
        matches = re.findall(r'[A-Z\*]\*[A-Z\*]', line)
        
        if len(matches) > 0:
            
            if '_Sp_' in rl[line_num-1]:
                if AA_file not in spar_stops:
                    spar_stops[AA_file] = 1
                else:
                    spar_stops[AA_file] += 1
                
            else:
                if AA_file not in scer_stops:
                    scer_stops[AA_file] = 1
                else:
                    scer_stops[AA_file] += 1
            
            if AA_file not in stops:
                stops[AA_file] = 1
            else:
                stops[AA_file] += 1
                
genes_to_drop = []

df = pd.DataFrame.from_dict(stops, orient='index')
df.columns = ['num_genes_with_stop_in_ORF']
print('genes with stop in ORF:', df.shape[0])

print()
print('genes with introns: ', df[df.index.str.strip('.faa').isin(intron_genes)].shape[0])
genes_to_drop += list(df[df.index.str.strip('.faa').isin(intron_genes)].index)
df = df[~df.index.str.strip('.faa').isin(intron_genes)]
print('genes after dropping genes with introns: ', df.shape[0])
print()

print('genes with less than 1 strain with stop in ORF:', df[df['num_genes_with_stop_in_ORF'] <= 1].shape[0])
print('genes with less than 5 strains with stop in ORF:', df[df['num_genes_with_stop_in_ORF'] <= 5].shape[0])
print('genes with less than 10 strains with stop in ORF:', df[df['num_genes_with_stop_in_ORF'] <= 10].shape[0])
genes_to_drop += list(df[df['num_genes_with_stop_in_ORF'] > 10].index)

print()
print('number of genes to drop: ', len(genes_to_drop))

genes with stop in ORF: 2996

genes with introns:  83
genes after dropping genes with introns:  2913

genes with less than 1 strain with stop in ORF: 749
genes with less than 5 strains with stop in ORF: 1698
genes with less than 10 strains with stop in ORF: 2067

number of genes to drop:  929


In [3]:
#genes with only stops in Spar ORF (not Scer ORF)

print('genes with only stops in Spar ORFs: ', len([i for i in spar_stops.keys() if i not in scer_stops.keys()]))
print('genes with only stops in Scer ORFs: ', len([i for i in scer_stops.keys() if i not in spar_stops.keys()]))
print('genes with stops in ORFs of both species: ', len([i for i in scer_stops.keys() if i in spar_stops.keys()]))

genes with only stops in Spar ORFs:  58
genes with only stops in Scer ORFs:  2623
genes with stops in ORFs of both species:  315


In [4]:
#are we dropping any candidate genes? no

gene_dict = {'YLR397C':'AFG2',
             'YGR098C':'ESP1',
             'YMR168C':'CEP3',
             'YKR054C': 'DYN1',
             'YHR023W':'MYO1',
             'YDR180W':'SCC2',
             'YPL174C':'NIP100',
             'YCR042C': 'TAF2',
             'YMR016C':'SOK2',
             'YJR135C':'MCM22',
             'YJL025W':'RRN7',
             'YDR443C':'SSN2',
             'YKL134C':'OCT1',
             'YPR164W':'MMS1',}

[i for i in gene_dict.keys() if i in genes_to_drop]

[]

In [42]:
files_to_edit = stops.keys()

In [63]:
#make new files - drop strains with stop codons

from shutil import copyfile

AA_dir = 'alignments/1011Scer_allSpar_codon_alignments/AA_unaligned/'
nuc_dir = 'alignments/1011Scer_allSpar_codon_alignments/nuc_unaligned/'
AA_outdir = 'alignments/1011Scer_allSpar_codon_alignments/AA_unaligned_clean/'
nuc_outdir = 'alignments/1011Scer_allSpar_codon_alignments/nuc_unaligned_clean/'

for AA_file in os.listdir(AA_dir):
    
    gene = AA_file.strip('.faa')
    nuc_file = AA_file.replace('.faa', '.fna')
    
    if  not os.path.exists(nuc_dir+nuc_file):
        continue
    
    if gene in genes_to_drop:
        continue
        
    if AA_file not in files_to_edit:
        copyfile(AA_dir+AA_file, AA_outdir+AA_file)
        copyfile(nuc_dir+nuc_file, nuc_outdir+nuc_file)
        continue
    
    
    strains_to_drop = []
    to_write = ''
    for record in SeqIO.parse(AA_dir+AA_file,'fasta'):
        if '*' in record.seq[:-1]:
            strains_to_drop += [record.id]
            continue
        to_write += '>{}\n{}\n'.format(record.id, str(record.seq))
    
    AA_out = open(AA_outdir+AA_file, 'w')
    AA_out.write(to_write)
    AA_out.close()

    
    to_write=''
    for record in SeqIO.parse(nuc_dir+nuc_file,'fasta'):
        if record.id not in strains_to_drop: 
            to_write += '>{}\n{}\n'.format(record.id, str(record.seq))
    
    nuc_out = open(nuc_outdir+nuc_file, 'w')
    nuc_out.write(to_write)
    nuc_out.close()    

In [54]:
len(os.listdir(AA_dir)), len(os.listdir(nuc_dir))

(4883, 4882)

In [53]:
len(os.listdir(AA_outdir)), len(os.listdir(nuc_outdir))

(3953, 3953)

### Aligned AA seqs on Savio using MUSCLE

`#!/bin/bash`

`/global/home/users/clairedubin/programs/muscle -in $1 -out $2 -maxiters 2`

Sort AA and nucleotide files so strains are in the same order

In [6]:
#sort AA and nuc files so everything is in the same order

AA_dir = 'alignments/1011Scer_allSpar_codon_alignments/AA_aligned_clean/'
nuc_dir = 'alignments/1011Scer_allSpar_codon_alignments/nuc_unaligned_clean/'
AA_outdir = 'alignments/1011Scer_allSpar_codon_alignments/AA_aligned_clean_sorted/'
nuc_outdir = 'alignments/1011Scer_allSpar_codon_alignments/nuc_unaligned_clean_sorted/'

for AA_file in os.listdir(AA_dir):

    nuc_file = AA_file.replace('.muscle_afa', '.fna')
    
    if  not os.path.exists(nuc_dir+nuc_file):
        continue
    
    nuc_dict =  SeqIO.to_dict(SeqIO.parse(nuc_dir+nuc_file,'fasta'))
    to_write = ''
    for record in sorted(nuc_dict):
        
        to_write += '>{}\n{}\n'.format(record, str(nuc_dict[record].seq))

    nuc_out = open(nuc_outdir+nuc_file, 'w')
    nuc_out.write(to_write)
    nuc_out.close()

    AA_dict =  SeqIO.to_dict(SeqIO.parse(AA_dir+AA_file,'fasta'))
    to_write = ''
    for record in sorted(AA_dict):
        
        to_write += '>{}\n{}\n'.format(record, str(AA_dict[record].seq))

    AA_out = open(AA_outdir+AA_file, 'w')
    AA_out.write(to_write)
    AA_out.close()

### Make codon alignments with PAL2NAL

In [2]:
#make codon alignments using pal2nal

AA_aligned = 'alignments/1011Scer_allSpar_codon_alignments/AA_aligned_clean_sorted/'
nuc_unaligned = 'alignments/1011Scer_allSpar_codon_alignments/nuc_unaligned_clean_sorted/'
codon_aligned = 'alignments/1011Scer_allSpar_codon_alignments/codons_aligned/'
log_file = 'alignments/1011Scer_allSpar_codon_alignments/pal2nal.log'

for i, f in enumerate(os.listdir(AA_aligned)):
    
    if os.path.exists(codon_aligned + f.replace('.muscle_afa', '.pal2nal.fasta')):
        continue
    
    if os.path.exists(nuc_unaligned+f.replace('.muscle_afa','.fna')):
    
        c = 'perl /Users/clairedubin/programs/pal2nal.v14/pal2nal.pl ' + AA_aligned+f + ' ' + nuc_unaligned+f.replace('.muscle_afa','.fna') + ' -output fasta -nogap > ' + codon_aligned + f.replace('.muscle_afa', '.pal2nal.fasta') + ' 2>' + log_file
    
        !{c}